In [1]:
%pip install transformers torch

You should consider upgrading via the '/Users/danilkladnitsky/.pyenv/versions/3.10.4/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
from transformers import GPT2LMHeadModel, AutoTokenizer
import torch
import re

class ChineseSentenceGenerator:
    def __init__(self, model_path: str, device: str = None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = GPT2LMHeadModel.from_pretrained(model_path)

        self.model.eval()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def generate(self, word: str, max_length: int = 60, num_return_sequences: int = 1) -> list:
        prompt = f"请用词语“{word}”造句："
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=max_length,
                do_sample=True,
                top_k=50,
                top_p=0.9,
                temperature=0.9,
                num_return_sequences=num_return_sequences,
                repetition_penalty=1.3,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )

        print(self.tokenizer.pad_token_id, self.tokenizer.eos_token_id)

        results = []
        for output in outputs:
            decoded = self.tokenizer.decode(output, skip_special_tokens=True)
            print(decoded)
            sentence = decoded.replace(prompt, "").replace(" ", "").strip()
            sentence = re.split(r"[。！？]", sentence)[0] + "。"  # stop at first punctuation
            results.append(sentence)

        return results
    
generator = ChineseSentenceGenerator("../models/hsk1-gpt2-jieba")

results = generator.generate("学生", max_length=60, num_return_sequences=3)
for i, sent in enumerate(results):
    print(f"📝 Sentence {i+1}: {sent}")

0 None
请 用 词语 “ 学生 ” 造句 : 很多 学生 去 了 。 这 本书 。 。 。 。 一个 写作 客气 。 。 ! 你 。 。 。 ” 一个 ? 。 。 。 。 。 我 。 ! 。 。 的 雨 。 。 。 。 。 本 。 。 。 。 。 ” 。
请 用 词语 “ 学生 ” 造句 : 很多 学生 买 了 这 本书 。 。 。 。 ! 一个 。 有 很会 。 。 人去 。 。 。 。 吗 ? ? ” 是 的 , 对 吗 ? , 不是 星期六 。 。 我 的 . 谢谢 。 ” 。 。 ” 。 ” 。 。 ” 。
请 用 词语 “ 学生 ” 造句 : 有个 学生 想见 你 。 了 一个 人 ! ? 。 。 。 ” 一个 去 一下 。 。 。 是 的 , 我 不是 你 的 朋友 。 。 我 很 高兴 。 来 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。 。
📝 Sentence 1: 请用词语“学生”造句:很多学生去了。
📝 Sentence 2: 请用词语“学生”造句:很多学生买了这本书。
📝 Sentence 3: 请用词语“学生”造句:有个学生想见你。
